In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


def extract_insta_data(user_id="user_id", user_passwd="user_passwd", wish_num=3,
                       login_option="instagram",
                       driver_path="chrome_driver_path", keyword="Wheat", 
                       instagram_id_name="username", instagram_pw_name="password",
                       instagram_login_btn=".sqdOP.L3NKy.y3zKF     ",
                       first_img_css="div.v1Nh3.kIKUG._bz0w",
                       location_object_css="div.o-MQd.z8cbW > div.M30cS > div.JF9hh > div._7UhW9.PIoXz.MMzan._0PwGv.fDxYl.T0kll > a.O4GlU",
                       upload_id_object_css="div.e1e1d > div._7UhW9.xLCgt.qyrsm.KV-D4.uL8Hv.T0kll > span.Jv7Aj.MqpiF  > a.sqdOP.yWX7d._8A5w5.ZIAjV ",
                       date_object_css="div.qF0y9.Igw0E.IwRSH.eGOV_.acqo5._4EzTm.pjcA_.aGBdT                                                   > div._7UhW9.PIoXz.MMzan._0PwGv.uL8Hv          > time.FH9sR.RhOlS",
                       main_text_object_css="div.C7I1f.X7jCj > div.C4VMK > div.MOdxS > span",
                       tag_css=".C7I1f.X7jCj",
                       comment_more_btn="div.qF0y9.Igw0E.IwRSH.eGOV_.ybXk5._4EzTm                                                                                                               > button.sqdOP.yWX7d.y3zKF     ",
                       comment_ids_objects_css="a.sqdOP.yWX7d._8A5w5.ZIAjV ",
                       comment_texts_objects_css="span._7UhW9.xLCgt.MMzan.KV-D4.se6yk.T0kll ",
                       print_flag=False,
                       next_arrow_btn_css1="div.l8mY4.feth3 > .wpO6b  ",
                       save_file_name="instagram_extract",
                       save_file_name_tag="instagram_tag"):
    '''
    아이디, 비밀번호, 검색키워드, 검색 게시글수 입력
    내용, 코멘트 추출하여 CSV파일로 저장 
    '''
    driver = wd.Chrome(driver_path)
    
    print(f"login start - option {login_option}")
    
    login_url = "https://www.instagram.com/accounts/login/"
    driver.get(login_url)
    time.sleep(8)
    
    is_login_success = False
    
    if login_option == "instagram":
        try:
            instagram_id_form = driver.find_element(By.CSS_SELECTOR, "#loginForm > div > div:nth-child(1) > div > label > input" )
            instagram_id_form.send_keys(user_id)
            time.sleep(4)

            instagram_pw_form = driver.find_element(By.CSS_SELECTOR,"#loginForm > div > div:nth-child(2) > div > label > input")
            instagram_pw_form.send_keys(user_passwd)
            time.sleep(6)
            
            login_ok_button = driver.find_element_by_css_selector(instagram_login_btn)
            login_ok_button.click()
            is_login_success = True
        except:
            print("instagram login fail")
            is_login_success = False

        time.sleep(8)
        
    if is_login_success:
        print(f"login {login_option} success")
        print(f"Start {keyword} Extract")

        url = "https://www.instagram.com/explore/tags/{}/".format(keyword)

        instagram_tags = []
        instagram_tag_dates = []

        driver.get(url)
        time.sleep(10)


        print("login success")

        # 첫번째 게시물
        driver.find_element_by_css_selector(first_img_css).click()


        # data lists
        location_infos = []
        location_hrefs = []

        upload_ids = []

        date_texts = []
        date_times = []
        date_titles = []

        main_texts = []

        instagram_tags = []

        comments = []

        check_arrow = True

        count_extract = 0
        cnt_num = 1

        while True:
            if count_extract >= wish_num:
                break
            time.sleep(5.0)
            # 위치정보

            if check_arrow == False:
                break

            try:
                location_object = driver.find_element_by_css_selector(location_object_css)
                location_info = location_object.text
                location_href = location_object.get_attribute("href")
            except:
                location_info = None
                location_href = None

            # 올린사람 ID
            try:
                upload_id_object = driver.find_element_by_css_selector(upload_id_object_css)
                upload_id = upload_id_object.text
            except:
                upload_id = None


            # 날짜
            try:
                date_object = driver.find_element_by_css_selector(date_object_css)
                date_text = date_object.text
                date_time = date_object.get_attribute("datetime")
                date_title = date_object.get_attribute("title")
            except:
                date_text = None
                date_time = None
                date_title = None


            # 본문
            try:
                main_text_object = driver.find_element_by_css_selector(main_text_object_css)
                main_text = main_text_object.text.replace("\n","").replace(",","")
            except:
                main_text = None


            ## 본문 속 태그
            try:
                data = driver.find_element_by_css_selector(tag_css) # C7I1f X7jCj
                tag_raw = data.text
                tags = re.findall('#[A-Za-z0-9가-힣]+', tag_raw) 
                tag = ''.join(tags).replace("#"," ") # "#" 제거

                tag_data = tag.split()

                for tag_one in tag_data:
                    instagram_tags.append(tag_one)
            except:
                continue


            # 댓글
            ## 더보기 버튼 클릭
            try:
                more_btn_list = driver.find_elements_by_css_selector(comment_more_btn)
                
                for more_btn in more_btn_list:
                    if '답글 보기' in more_btn.text:
                        more_btn.click()
                        time.sleep(1)
            except:
                print("----------------------fail to click more btn----------------------------------")
                continue

            ## 댓글 데이터
            try:
                comment_data = {}
                comment_ids_objects = driver.find_elements_by_css_selector(comment_ids_objects_css)
                comment_texts_objects = driver.find_elements_by_css_selector(comment_texts_objects_css)
                
                try:
                    for i in range(len(comment_ids_objects)):
                        comment_data[str((i+1))] = {"comment_id":comment_ids_objects[i].text, "comment_text":comment_texts_objects[i].text} 
                except:
                    print(f"총 검색량 {wish_num}개 중에 {cnt_num}번째 진행중")
                    cnt_num += 1



            except Exception as E:
                print(E)
                comment_id = None
                comment_text = None
                comment_data = {}


            try:
                if comment_data != {}:
                    keys = list(comment_data.keys())

                    for key in keys:
                        if comment_data[key]['comment_id'] == upload_id:
                            tags = re.findall('#[A-Za-z0-9가-힣]+', comment_data[key]['comment_text']) 
                            tag = ''.join(tags).replace("#"," ") # "#" 제거

                            tag_data = tag.split()

                            for tag_one in tag_data:
                                instagram_tags.append(tag_one)
            except:
                continue



            location_infos.append(location_info)
            location_hrefs.append(location_href)

            upload_ids.append(upload_id)

            date_texts.append(date_text)
            date_times.append(date_time)
            date_titles.append(date_title)

            main_texts.append(main_text)

            comment_json = json.dumps(comment_data, ensure_ascii=False)

            comments.append(comment_json.replace(","," "))
            

            if print_flag:
                print("location_info : ", location_info)
                print("location_href : ", location_href)
                print("upload id : ", upload_id)
                print("date : {} {} {}".format(date_text, date_time, date_title))
                print("main : ", main_text)
                print("comment : ", comment_data)

                print("insta tags : ", instagram_tags)

            try:
                WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, next_arrow_btn_css1)))
                time.sleep(5.0)
                next_arrow_btn = driver.find_element_by_css_selector(next_arrow_btn_css1)
                next_arrow_btn.send_keys(Keys.ENTER)
            except:
                check_arrow = False

            count_extract += 1

        try:
            # 본문과 코멘트 저장
            insta_info_df = pd.DataFrame({"main_text":main_texts})# , "comment":comments}) # 기타 정보 "location_info":location_infos, "location_href":location_hrefs, "upload_id":upload_ids, "date_text":date_texts, "date_time":date_times, "date_title":date_titles, 
            insta_info_df.to_csv("{}.csv".format(save_file_name), index=False)
        except:
            print("fail to save data")


        try:
            # 태그 저장
            insta_tag_df = pd.DataFrame({"tag":instagram_tags})
            insta_tag_df.to_csv("{}.csv".format(save_file_name_tag), index=False)
        except:
            print("fail to save tag data")        
    elif not is_login_success:
        print(f"login {login_option} fail")
    print("종료")
    driver.close()
    driver.quit()

In [5]:
id = input("전화번호 or 이메일: ")
pw = input("인스타 비밀번호: ")
# id_name = input("인스타 아이디: ")
my_keyword = input("검색어")
how_many = int(input("검색할 게시글 개수"))
my_driver_path = "c:/pydata/chromedriver.exe"

if __name__ == "__main__":
    extract_insta_data(user_id=id, user_passwd=pw, wish_num=how_many,
                       login_option="instagram",
                       driver_path=my_driver_path, keyword=my_keyword, 
                       instagram_id_name= id, instagram_pw_name=pw,
                       instagram_login_btn=".sqdOP.L3NKy.y3zKF     ",
                       first_img_css="div.v1Nh3.kIKUG._bz0w",
                       location_object_css="div.o-MQd.z8cbW > div.M30cS > div.JF9hh > div._7UhW9.PIoXz.MMzan._0PwGv.fDxYl.T0kll > a.O4GlU",
                       upload_id_object_css="div.e1e1d > div._7UhW9.xLCgt.qyrsm.KV-D4.uL8Hv.T0kll > span.Jv7Aj.MqpiF  > a.sqdOP.yWX7d._8A5w5.ZIAjV ",
                       date_object_css="div.qF0y9.Igw0E.IwRSH.eGOV_.acqo5._4EzTm.pjcA_.aGBdT                                                   > div._7UhW9.PIoXz.MMzan._0PwGv.uL8Hv          > time.FH9sR.RhOlS",
                       main_text_object_css="div.C7I1f.X7jCj > div.C4VMK > div.MOdxS > span",
                       tag_css=".C7I1f.X7jCj",
                       comment_more_btn="div.qF0y9.Igw0E.IwRSH.eGOV_.ybXk5._4EzTm                                                                                                               > button.sqdOP.yWX7d.y3zKF     ",
                       comment_ids_objects_css="a.sqdOP.yWX7d._8A5w5.ZIAjV ",
                       comment_texts_objects_css="span._7UhW9.xLCgt.MMzan.KV-D4.se6yk.T0kll ",
                       print_flag=False,
                       next_arrow_btn_css1="div.l8mY4.feth3 > .wpO6b  ",
                       save_file_name="../data/감성분석/crawling/instagram_"+my_keyword,
                       save_file_name_tag="../data/감성분석/crawling/instagram_tag_"+my_keyword)

login start - option instagram
login instagram success
Start 포도 Extract
login success
총 검색량 3개 중에 1번째 진행중
총 검색량 3개 중에 2번째 진행중
총 검색량 3개 중에 3번째 진행중
종료
